# Introduction and Pre-Processing

In this notebook, we demonstrate key preprocessing steps for dealing with image data in BART. More precisely, we will reconstruct a slice of multi-coil 3D data.

## TODO:
   * [ ] Flesh out descriptions - current ones are just placeholders for myself
   * [ ] What do view params do?

In this tutorial, we will be exploring reconstructing a 3D FSE scan of a knee from Raw K-Space data. The data is available [here](http://old.mridata.org/fullysampled/knees) if you are looking to recreate these results on your own.

Before viewing the image, we are going to use `bart show` to get a better understanding of the data we are working with. BART data is stored in two files, a header (`.hdr`) file which contains metadata (dimensions, file type, etc) and a complex float (`.cfl`) file which contains the raw data. By default, `show` will print the raw complex values. With the `-m` flag, we can view the file metadata, as we've done below:

In [3]:
bart show -m kspace

Type: complex float
Dimensions: 16
AoD:	320	320	256	8	1	1	1	1	1	1	1	1	1	1	1	1


The `AoD` header tells us that we are working with k-space data of 320 values in the spatial frequency/$k_x$ direction, and 320 values in the phase encoding/$k_y$ direction. We have 256 slices, and the data has been collected from 8 coils.

[view](https://www.github.com/mrirecon/view) is an open-source viewer for BART data. We are going to use it to explore the 3D K-Space data and then extract a slice to reconstruct.

In [4]:
view kspace&

[1] 2029


**TODO**: What do BART sliders parameterize? Indicate here how users can recreate results, but explain what settings are doing. Additionally, would showing how `view` lets a user view the phase be of interest here as well?

**SETTINGS**
First slider: 0
Second: 0.001
Third 1.4
Fourth 1.0

We are going to extract the slice 128. Using the column on the left (for those following along: click the settings icon) we can choose which slice to view. We can adjust the value below this one to choose the coil to view the results from. 

In [5]:
bart slice -h

dbus[2029]: Dynamic session lookup supported but failed: launchd did not provide a socket path, verify that org.freedesktop.dbus-session.plist is loaded!
Usage: slice dim1 pos1 ... dimn posn <input> <output>

Extracts a slice from positions along dimensions.


-h		help


Recalling our image metadata, we can see that dimension `2` corresponds to the slices of the 3D image. We are going to extract the slice at position `128`, so our command looks as follows:

In [6]:
bart slice 2 128 kspace ksp_slice

Using `show` we verify that our slice dimensions look correct.

In [7]:
bart show -m ksp_slice

Type: complex float
Dimensions: 16
AoD:	320	320	1	8	1	1	1	1	1	1	1	1	1	1	1	1


Dimensions look good! We have preserved the $k_x$ and $k_y$ dimensions as well as the number of coils. Let's look at the image in `view`

In [8]:
view ksp_slice&

[2] 2035


: 1

We will now use BART's build in coil compression method, `cc`, to compress to four virtual coils. The `-G` flag enables [Geometric](https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.24267) coil compression, but additional flags offer SVD and ESPIRiT based methods as well.

In [10]:
bart cc -p 4 -G ksp_slice ksp_cc

Done.


`-p` allows us to specify the number of virtual coils we wish to compress to.

In [11]:
bart show -m ksp_cc

Type: complex float
Dimensions: 16
AoD:	320	320	1	4	1	1	1	1	1	1	1	1	1	1	1	1


Dimension `3` - the number of coils - has changed to reflect our compression. This looks good, let's take a look at the image in `view`.

In [12]:
view ksp_cc&

[3] 2159


Now, we are going to demonstrate how to perform a simple inverse FFT on the k-space data. Note: to specify the dimensions BART operates on, one must pass in an appropriate bitmask: (e.g., for dimensions 0 and 1, this bitmask is calculated as $2^0 + 2^1 = 3$). `bart bitmask` automatically computes this value for us.

`-u` and `-i` indicate that we want a unitary and inverse FFT respectively.

In [14]:
bart fft -h

Usage: fft [-u] [-i] [-n] bitmask <input> <output>

Performs a fast Fourier transform (FFT) along selected dimensions.

-u		unitary
-i		inverse
-n		un-centered
-h		help


In [15]:
bart fft -u -i $(bart bitmask 0 1) ksp_cc ksp_fft

In [16]:
bart show -m ksp_fft

Type: complex float
Dimensions: 16
AoD:	320	320	1	4	1	1	1	1	1	1	1	1	1	1	1	1


In [17]:
view ksp_fft&

[4] 2167


: 1

Voila! The knee image has been reconstructed across the four channels. We can now use a root-sum-of-squares (`rss`) method to combine the multi-coil data.

In [19]:
bart rss $(bart bitmask 3) ksp_fft ksp_combined

In [22]:
view ksp_combined&

[1] 2174


: 1